<p>which python<p>
<p>source .venv/bin/activate<p>
<p>jupyter notebook --no-browser --ip=0.0.0.0<p>

In [ ]:
!nvidia-smi

In [ ]:
import torch
print(torch.cuda.is_available())

# STEP 1: Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import cv2
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")
torch.manual_seed(0)

# STEP 2: Load and Explore the Dataset

In [ ]:
# Load the CSV files
print("Loading dataset...")
train_df = pd.read_csv('data/Train.csv')
test_df = pd.read_csv('data/Test.csv')
meta_df = pd.read_csv('data/Meta.csv')

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Meta data shape: {meta_df.shape}")

# Display first few rows
print("\nTraining data sample:")
print(train_df.head())

print("\nTest data sample:")
print(test_df.head())

print("\nMeta data sample:")
print(meta_df.head())

# STEP 3: Data Analysis and Visualization

In [ ]:
# Check unique classes in training data
unique_classes = sorted(train_df['ClassId'].unique())
print(f"\nUnique classes in training: {unique_classes}")
print(f"Number of classes: {len(unique_classes)}")

# Class distribution analysis
plt.figure(figsize=(15, 5))

# Class distribution
plt.subplot(1, 3, 1)
class_counts = train_df['ClassId'].value_counts().sort_index()
plt.bar(class_counts.index, class_counts.values)
plt.title('Class Distribution in Training Data')
plt.xlabel('Class ID')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)

# Image dimensions analysis
plt.subplot(1, 3, 2)
plt.scatter(train_df['Width'], train_df['Height'], alpha=0.6)
plt.xlabel('Width')
plt.ylabel('Height')
plt.title('Image Dimensions Distribution')

# ROI (Region of Interest) analysis
plt.subplot(1, 3, 3)
roi_width = train_df['Roi.X2'] - train_df['Roi.X1']
roi_height = train_df['Roi.Y2'] - train_df['Roi.Y1']
plt.scatter(roi_width, roi_height, alpha=0.6)
plt.xlabel('ROI Width')
plt.ylabel('ROI Height')
plt.title('ROI Dimensions Distribution')

plt.tight_layout()
plt.show()

# STEP 4: Custom Dataset Class

In [ ]:
class ArtStyleDataset(Dataset):
    """
    Custom PyTorch Dataset for Art Style Classification
    
    This class handles:
    - Loading images from file paths
    - Applying ROI cropping if specified
    - Applying transformations (augmentation)
    - Converting to tensors
    """
    
    def __init__(self, dataframe, root_dir, transform=None, use_roi=True):
        """
        Args:
            dataframe: pandas DataFrame with image paths and labels
            root_dir: Root directory containing images
            transform: Optional transform to be applied on images
            use_roi: Whether to use ROI cropping
        """
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.use_roi = use_roi
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Get image path and label
        row = self.dataframe.iloc[idx]
        img_path = os.path.join(self.root_dir, row['Path'])
        label = row['ClassId']
        
        # Load image
        try:
            image = cv2.imread(img_path)
            if image is None:
                raise ValueError(f"Could not load image: {img_path}")
            
            # Convert BGR to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Apply ROI cropping if specified
            if self.use_roi and all(col in row for col in ['Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2']):
                x1, y1, x2, y2 = int(row['Roi.X1']), int(row['Roi.Y1']), int(row['Roi.X2']), int(row['Roi.Y2'])
                image = image[y1:y2, x1:x2]
            
            # Convert to PIL Image for transforms
            image = Image.fromarray(image)
            
            # Apply transforms
            if self.transform:
                image = self.transform(image)
            
            return image, label
            
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a black image if there's an error
            if self.transform:
                image = self.transform(Image.new('RGB', (224, 224), (0, 0, 0)))
            else:
                image = torch.zeros(3, 224, 224)
            return image, label

# STEP 5: Data Transforms and Augmentation

In [ ]:
# Define transforms for training (with augmentation)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define transforms for validation/test (no augmentation)
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Data transforms defined:")
print("- Training: Resize, Rotation, Flip, ColorJitter, Normalize")
print("- Validation: Resize, Normalize only")

# STEP 6: Create Data Loaders

In [ ]:
# Filter training data to only include classes 0-41 (42 classes total)
# This leaves class 42 as OOD (Out-of-Distribution)
train_df_filtered = train_df[train_df['ClassId'] <= 41].copy()

print(f"Original training samples: {len(train_df)}")
print(f"Filtered training samples (classes 0-41): {len(train_df_filtered)}")
print(f"Class 42 samples excluded: {len(train_df[train_df['ClassId'] == 42])}")

# Split filtered training data into train and validation sets
train_data, val_data = train_test_split(
    train_df_filtered,  # Use filtered dataframe instead of train_df
    test_size=0.2, 
    random_state=42, 
    stratify=train_df_filtered['ClassId']  # Use filtered dataframe for stratification
)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

# Create datasets
train_dataset = ArtStyleDataset(train_data, 'data', transform=train_transforms, use_roi=True)
val_dataset = ArtStyleDataset(val_data, 'data', transform=val_transforms, use_roi=True)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print(f"Batch size: {batch_size}")
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

# STEP 7: Visualize Sample Data

In [ ]:
def imshow(tensor, title=None):
    """Display a tensor as an image"""
    # Denormalize the image
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    image = tensor.clone().detach()
    image = image.numpy().transpose((1, 2, 0))
    image = std * image + mean
    image = np.clip(image, 0, 1)
    
    plt.imshow(image)
    if title:
        plt.title(title)
    plt.axis('off')

def visualize_batch(data_loader, num_samples=8):
    """Visualize a batch of images"""
    data_iter = iter(data_loader)
    images, labels = next(data_iter)
    
    plt.figure(figsize=(12, 6))
    for i in range(min(num_samples, len(images))):
        plt.subplot(2, 4, i + 1)
        imshow(images[i], title=f'Class {labels[i].item()}')
    plt.tight_layout()
    plt.show()

# Visualize training samples
print("Sample training images:")
visualize_batch(train_loader)

# STEP 8: Model Architecture

In [ ]:
class ArtStyleClassifier(nn.Module):
    """
    CNN model for art style classification using transfer learning
    
    Architecture:
    - Pre-trained ResNet50 as backbone
    - Custom classifier head
    - Dropout for regularization
    """
    
    def __init__(self, num_classes, pretrained=True):
        super(ArtStyleClassifier, self).__init__()
        
        # Load pre-trained ResNet50
        self.backbone = models.resnet50(pretrained=pretrained)
        
        # Get the number of features from the last layer
        num_features = self.backbone.fc.in_features
        
        # Replace the final layer with custom classifier
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.backbone(x)

# Alternative: EfficientNet model
class EfficientNetClassifier(nn.Module):
    """
    Alternative model using EfficientNet
    """
    
    def __init__(self, num_classes, pretrained=True):
        super(EfficientNetClassifier, self).__init__()
        
        # Load pre-trained EfficientNet
        self.backbone = models.efficientnet_b0(pretrained=pretrained)
        
        # Get the number of features
        num_features = self.backbone.classifier[1].in_features
        
        # Replace classifier
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        return self.backbone(x)

# Update unique_classes to reflect only the classes we're training on
unique_classes = sorted(train_df_filtered['ClassId'].unique())
print(f"\nUnique classes in training: {unique_classes}")
print(f"Number of classes: {len(unique_classes)}")

# Create model
num_classes = len(unique_classes)  # This will be 42 instead of 43
model = ArtStyleClassifier(num_classes, pretrained=True)
# model = EfficientNetClassifier(num_classes, pretrained=True)

# Move model to device
model = model.to(device)

# Display model architecture
print(f"Model created with {num_classes} classes")
print(f"Model moved to {device}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# STEP 9: Loss Function and Optimizer

In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

print("Training setup:")
print(f"Loss function: CrossEntropyLoss")
print(f"Optimizer: Adam (lr=0.001, weight_decay=1e-4)")
print(f"Scheduler: StepLR (step_size=10, gamma=0.1)")

# STEP 10: Training Functions

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """
    Train the model for one epoch
    
    Returns:
        Average loss and accuracy for the epoch
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc='Training')
    
    for batch_idx, (images, labels) in enumerate(progress_bar):
        images, labels = images.to(device), labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Update progress bar
        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Acc': f'{100.*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

def validate_epoch(model, val_loader, criterion, device):
    """
    Validate the model for one epoch
    
    Returns:
        Average loss, accuracy, and F1 score for the epoch
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc='Validation')
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Store predictions for F1 score calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            # Update progress bar
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100.*correct/total:.2f}%'
            })
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    
    # Calculate F1 score
    f1_macro = f1_score(all_labels, all_predictions, average='macro')
    f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
    
    return epoch_loss, epoch_acc, f1_macro, f1_weighted

# STEP 11: Training Loop

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=50):
    """
    Complete training loop with validation
    
    Returns:
        Training history dictionary
    """
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1_macro': [],
        'val_f1_weighted': []
    }
    
    best_f1 = 0.0
    best_model_state = None
    
    print(f"Starting training for {num_epochs} epochs...")
    print("-" * 50)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        # Training
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        
        # Validation
        val_loss, val_acc, val_f1_macro, val_f1_weighted = validate_epoch(model, val_loader, criterion, device)
        
        # Update learning rate
        scheduler.step()
        
        # Save best model
        if val_f1_macro > best_f1:
            best_f1 = val_f1_macro
            best_model_state = model.state_dict().copy()
            print(f"New best F1 score: {best_f1:.4f}")
        
        # Store history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1_macro'].append(val_f1_macro)
        history['val_f1_weighted'].append(val_f1_weighted)
        
        # Print epoch results
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        print(f'Val F1 (Macro): {val_f1_macro:.4f}, Val F1 (Weighted): {val_f1_weighted:.4f}')
        print(f'Current LR: {optimizer.param_groups[0]["lr"]:.6f}')
        print("-" * 50)
        
        # Early stopping check
        if epoch > 10 and val_f1_macro < max(history['val_f1_macro'][-10:]) - 0.01:
            print("Early stopping triggered!")
            break
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Best model loaded with F1 score: {best_f1:.4f}")
    
    return history

In [ ]:
# Start training
history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20)

# STEP 12: Training Visualization

In [ ]:
def plot_training_history(history):
    """
    Plot training history
    """
    plt.figure(figsize=(15, 5))
    
    # Loss plot
    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Accuracy plot
    plt.subplot(1, 3, 2)
    plt.plot(history['train_acc'], label='Training Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)
    
    # F1 Score plot
    plt.subplot(1, 3, 3)
    plt.plot(history['val_f1_macro'], label='F1 Macro')
    plt.plot(history['val_f1_weighted'], label='F1 Weighted')
    plt.title('F1 Scores')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Plot training history
plot_training_history(history)

# STEP 13: Model Evaluation

In [ ]:
def evaluate_model(model, val_loader, device):
    """
    Comprehensive model evaluation
    """
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Evaluating'):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    # Calculate metrics
    f1_macro = f1_score(all_labels, all_predictions, average='macro')
    f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
    
    print(f"Final F1 Score (Macro): {f1_macro:.4f}")
    print(f"Final F1 Score (Weighted): {f1_weighted:.4f}")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_predictions))
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    plt.figure(figsize=(12, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    return f1_macro, f1_weighted, all_probabilities

In [ ]:
# Evaluate the model
f1_macro, f1_weighted, val_probabilities = evaluate_model(model, val_loader, device)

# STEP 14: Test Data Prediction

In [ ]:
def predict_test_data(model, test_df, root_dir, device, batch_size=32):
    """
    Make predictions on test data
    """
    # Create test dataset
    test_dataset = ArtStyleDataset(test_df, root_dir, transform=val_transforms, use_roi=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.eval()
    all_predictions = []
    all_probabilities = []
    
    with torch.no_grad():
        for images, _ in tqdm(test_loader, desc='Predicting'):
            images = images.to(device)
            
            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    return np.array(all_predictions), np.array(all_probabilities)

In [ ]:
# Make predictions on test data
print("Making predictions on test data...")
test_predictions, test_probabilities = predict_test_data(model, test_df, 'data', device)

# STEP 15: Handle Unknown Class (11th Class)

In [ ]:
def detect_unknown_class(predictions, probabilities, threshold=0.7):
    """
    Detect potential unknown class based on prediction confidence
    
    Args:
        predictions: Predicted class labels
        probabilities: Prediction probabilities for each class
        threshold: Confidence threshold below which we classify as unknown
    
    Returns:
        Modified predictions with unknown class
    """
    # Get maximum probability for each prediction
    max_probabilities = np.max(probabilities, axis=1)
    
    # Create a copy of predictions
    modified_predictions = predictions.copy()
    
    # Mark low-confidence predictions as unknown class (assuming class 10 is unknown)
    unknown_mask = max_probabilities < threshold
    modified_predictions[unknown_mask] = 10  # Unknown class
    
    print(f"Detected {np.sum(unknown_mask)} potential unknown class samples")
    print(f"Percentage of unknown samples: {np.sum(unknown_mask)/len(predictions)*100:.2f}%")
    
    return modified_predictions

In [ ]:
# Detect unknown class samples
final_predictions = detect_unknown_class(test_predictions, test_probabilities, threshold=0.6)

# STEP 16: Create Submission File

In [ ]:
def create_submission(test_df, predictions, output_file='submission.csv'):
    """
    Create submission file in required format
    """
    submission_df = test_df.copy()
    submission_df['PredictedClassId'] = predictions
    
    # Save submission file
    submission_df[['Path', 'PredictedClassId']].to_csv(output_file, index=False)
    
    print(f"Submission file saved as {output_file}")
    
    # Display prediction distribution
    pred_counts = pd.Series(predictions).value_counts().sort_index()
    print("\nPrediction distribution:")
    print(pred_counts)
    
    return submission_df

In [ ]:
# Create submission file
submission = create_submission(test_df, final_predictions, output_file="submissionOOD.csv")

# STEP 17: Model Saving and Loading

In [ ]:
# Save the trained model
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'num_classes': num_classes,
    'f1_score': f1_macro,
    'history': history
}, 'art_style_classifierOOD.pth')

print("Model saved as 'art_style_classifierOOD.pth'")

In [ ]:
# Function to load the model
def load_model(model_path, num_classes):
    """
    Load a saved model
    """
    model = ArtStyleClassifier(num_classes)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    
    print(f"Model loaded from {model_path}")
    print(f"Best F1 score: {checkpoint['f1_score']:.4f}")
    
    return model

# Example of loading the model
# loaded_model = load_model('art_style_classifier.pth', num_classes)

# STEP 18: Advanced Features (Optional)

In [ ]:
# Model ensemble example
def create_ensemble_predictions(models, test_loader, device):
    """
    Create ensemble predictions from multiple models
    """
    all_predictions = []
    
    for model in models:
        model.eval()
        model_predictions = []
        
        with torch.no_grad():
            for images, _ in test_loader:
                images = images.to(device)
                outputs = model(images)
                probabilities = F.softmax(outputs, dim=1)
                model_predictions.append(probabilities.cpu().numpy())
        
        all_predictions.append(np.concatenate(model_predictions))
    
    # Average predictions
    ensemble_predictions = np.mean(all_predictions, axis=0)
    final_predictions = np.argmax(ensemble_predictions, axis=1)
    
    return final_predictions, ensemble_predictions

# Gradient-weighted Class Activation Mapping (Grad-CAM)
def generate_gradcam(model, image, class_idx, target_layer):
    """
    Generate Grad-CAM heatmap for model interpretability
    """
    model.eval()
    
    # Forward pass
    features = []
    def hook_fn(module, input, output):
        features.append(output)
    
    handle = target_layer.register_forward_hook(hook_fn)
    
    image = image.unsqueeze(0).to(device)
    output = model(image)
    
    # Backward pass
    model.zero_grad()
    output[0, class_idx].backward()
    
    # Get gradients and features
    gradients = target_layer.weight.grad
    activations = features[0]
    
    # Generate heatmap
    weights = torch.mean(gradients, dim=(2, 3))
    heatmap = torch.zeros(activations.shape[2:]).to(device)
    
    for i in range(weights.shape[1]):
        heatmap += weights[0, i] * activations[0, i]
    
    heatmap = F.relu(heatmap)
    heatmap = heatmap / torch.max(heatmap)
    
    handle.remove()
    
    return heatmap.cpu().numpy()

# STEP 19: Summary and Results

In [ ]:
# Updated load_model function to return history
def load_model_with_history(model_path, num_classes):
    """
    Load a saved model along with its training history
    """
    model = ArtStyleClassifier(num_classes)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    
    # Extract history if available
    history = checkpoint.get('history', None)
    f1_score = checkpoint.get('f1_score', 0.0)
    
    print(f"Model loaded from {model_path}")
    print(f"Best F1 score: {f1_score:.4f}")
    
    return model, history

# Option to load model instead of training
USE_SAVED_MODEL = True  # Set to True if you want to load a saved model

if USE_SAVED_MODEL:
    # Load the saved model
    loaded_model, history = load_model_with_history('art_style_classifierOOD.pth', num_classes)
    model = loaded_model
    
    # If no history available, skip visualization or create dummy
    if history is None:
        print("No training history available - skipping training plots")
        # You can either skip plot_training_history() or create dummy data
        history = {
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'val_f1_macro': [],
            'val_f1_weighted': []
        }
else:
    # Use the freshly trained model (existing code continues)
    history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20)
    
# Make predictions on test data
print("Making predictions on test data...")
test_predictions, test_probabilities = predict_test_data(model, test_df, 'data', device)
# Continue with evaluation
f1_macro, f1_weighted, val_probabilities = evaluate_model(model, val_loader, device)
# Detect unknown class samples
final_predictions = detect_unknown_class(test_predictions, test_probabilities, threshold=0.6)

In [ ]:
print("\n" + "="*60)
print("TRAINING COMPLETE - SUMMARY")
print("="*60)
print(f"Final F1 Score (Macro): {f1_macro:.4f}")
print(f"Final F1 Score (Weighted): {f1_weighted:.4f}")
print(f"Number of test predictions: {len(final_predictions)}")
print(f"Model saved: art_style_classifier.pth")
print(f"Submission file: submission.csv")
print(f"Device used: {device}")
print(f"Total training time: {len(history['train_loss'])} epochs")

print("\nModel Architecture:")
print(f"- Base model: ResNet50 (pre-trained)")
print(f"- Number of classes: {num_classes}")
print(f"- Total parameters: {total_params:,}")

print("\nNext steps to improve performance:")
print("1. Try different architectures (EfficientNet, Vision Transformer)")
print("2. Experiment with different data augmentation techniques")
print("3. Implement test-time augmentation (TTA)")
print("4. Use model ensembling")
print("5. Fine-tune hyperparameters")
print("6. Implement advanced loss functions (Focal Loss, Label Smoothing)")
print("7. Use cross-validation for more robust evaluation")
print("8. Analyze model predictions with Grad-CAM")